In [1]:
import time
import nltk
from nltk.corpus import stopwords
#NLP
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.collocations import *
import string, re

#Model Data Prep
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

%matplotlib inline
np.random.seed(0)

## Combaine All Preprocess Data To get in one DataFrame

In [2]:
job1 = pd.read_csv('job_scrapped_preprocess.csv')
job2 = pd.read_csv('job_scrapped_preprocess_2.csv')

In [3]:
job1.shape,job2.shape

((58, 15), (963, 15))

In [4]:
final_df = job1.append(job2)
final_df = final_df.reset_index(drop=True)
final_df.shape

(1021, 15)

- Before Going Further you must See Part1 and Part2

In [5]:
final_df.head(3)

,Title,Company,Link,Review,Salary,Description,Rate_by,low,high,min,max,adjusted_salary,adjusted_yearly_salary,combined_text,Salary_Bins
0,Backend Developer,SatSure Analytics India Private Limited,https://in.indeed.com/company/SatSure-Analytic...,0.0,"₹6,00,000 a year",Responsibilities Include:\nCollaborating with ...,Yearly,"₹6,00,000 a year",0,600000.0,0.0,600000.0,600000.0,Backend Developer Responsibilities Include:\nC...,3
1,Java/J2EE Developer,Sminetech Solutions Pvt Ltd,https://in.indeed.com/rc/clk?cmp=Sminetech-Sol...,0.0,"₹3,50,000 - ₹5,00,000 a year",Your Responsibilities:\nWrite maintainable/sca...,Yearly,"₹3,50,000","₹5,00,000 a year",350000.0,500000.0,425000.0,425000.0,Java/J2EE Developer Your Responsibilities:\nWr...,2
2,JavaScript Developer(WEB-UI),eifer software solutions,https://in.indeed.com/company/eifer-software-s...,0.0,"₹4,00,000 - ₹10,00,000 a year",JavaScript Developer(WEB-UI)\nLooking for a fr...,Yearly,"₹4,00,000","₹10,00,000 a year",400000.0,1000000.0,700000.0,700000.0,JavaScript Developer(WEB-UI) JavaScript Develo...,3


In [6]:
final_df.columns

Index(['Title', 'Company', 'Link', 'Review', 'Salary', 'Description',
       'Rate_by', 'low', 'high', 'min', 'max', 'adjusted_salary',
       'adjusted_yearly_salary', 'combined_text', 'Salary_Bins'],
      dtype='object')

- Here I am only deal with 'combined_text' and 'Salary_Bins' . combined_textis my X and target is my Salary_Bins

In [7]:
#Categorical target
X = final_df['combined_text']
y = final_df['Salary_Bins'].astype('category')

## Here I Use TfidfVectorizer to convert X text data to Numeric Data

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
#This works for individual job description, but how to save the model? 
tdidf = TfidfVectorizer(sublinear_tf = True, min_df=0.01, max_df=0.5, ngram_range=(1,3), stop_words='english')
fitted_vectorizer = tdidf.fit(X)
X_vect = fitted_vectorizer.transform(X)

In [9]:
X.shape,y.shape,X_vect.shape

((1021,), (1021,), (1021, 7309))

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_vect, y, test_size=0.25, random_state=4)

## Model Build

In [11]:
from sklearn.metrics import classification_report

In [12]:
def classifier(model,X_train, y_train,X_test,y_test):
    model = model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(classification_report(y_pred,y_test))
    
    return model

In [13]:
from sklearn.linear_model import SGDClassifier
model = SGDClassifier(alpha=0.0001, max_iter=1000, n_jobs=3)
model = classifier(model,X_train,y_train,X_test,y_test)

              precision    recall  f1-score   support

           0       0.73      0.68      0.71        44
           1       0.70      0.55      0.62        60
           2       0.51      0.61      0.56        33
           3       0.58      0.60      0.59        58
           4       0.56      0.53      0.54        36
           5       0.57      0.80      0.67        25

    accuracy                           0.61       256
   macro avg       0.61      0.63      0.61       256
weighted avg       0.62      0.61      0.61       256



## Test The Model

In [14]:
#load the model from disk
job="Data Entry"
# tfidf_vectorizer = pickle.load(open('fitted_vectorizer.pickle','rb'))
result = model.predict(fitted_vectorizer.transform([job]))

In [15]:
labels=['Below 50,000','50,000-70,000','70,000-90,000','90,000-120,000','120,000-150,000','150,000 and Above']
print(result)
print(labels[result[0]])

[0]
Below 50,000


## Model Dump For Production

In [16]:
#save the vectorizer to disk
pickle.dump(fitted_vectorizer, open("fitted_vectorizer.pickle", "wb"))

In [17]:
#save the machine model using joblib
from joblib import dump, load
dump(model, 'sgdclassifier.joblib')

['sgdclassifier.joblib']

In [18]:
clf = load('sgdclassifier.joblib')

In [19]:
#load the model from disk
job="Data Entry"
tfidf_vectorizer = pickle.load(open('fitted_vectorizer.pickle','rb'))
result = clf.predict(tfidf_vectorizer.transform([job]))

In [20]:
labels=['Below 50,000','50,000-70,000','70,000-90,000','90,000-120,000','120,000-150,000','150,000 and Above']
print(result)
print(labels[result[0]])

[0]
Below 50,000
